In [1]:
import numpy as np
import math

In [ ]:
class ThreeLayerConvNet:
    """
    A three-layer convolutional network with the following architecture:

    conv - relu - 2x2 max pool - affine - relu - affine - softmax

    The network operates on minibatches of data that have shape (N, C, H, W)
    consisting of N images, each with height H and width W and with C input
    channels.
    """
        
    def __init__(self, input_dim=(3, 32, 32), filter_num=32, filter_size=7, 
                 pad=0, stride=1, pool_size=2, hidden_dim=100, num_classes=10, 
                 weight_scale=1e-3, reg=0.0, dtype=np.float32):
        self.params = {}
        self.filter_num = filter_num
        self.filter_size = filter_size
        self.pad = pad
        self.stride = stride
        self.pool_size = pool_size
        self.reg = reg
        self.dtype = dtype
        
        W1 = np.random.randn(filter_num, input_dim[0], filter_size, filter_size) * weight_scale
        b1 = np.zeros((filter_num,))
        
        feature_map_h = (input_dim[1] - filter_size + 2*pad)/stride + 1 # H2 = (H1-F+2*P)/S+1
        feature_map_w = (input_dim[2] - filter_size + 2*pad)/stride + 1 # W2 = (W1-F+2*P)/S+1
        
        if not feature_map_h//1 == feature_map_h or not feature_map_w//1 == feature_map_w: # W2, H2 should be int
            raise ValueError('feature map width and height must be int, adjust filter size, padding or stride.')
        
        self.feature_map_h, self.feature_map_w = int(feature_map_h), int(feature_map_w)
        
        self.pooled_feature_map_h = math.ceil(feature_map_h/pool_size)
        self.pooled_feature_map_w = math.ceil(feature_map_h/pool_size)
        
        W2 = np.random.randn(self.pooled_feature_map_h * self.pooled_feature_map_w * filter_num, hidden_dim) \
             * weight_scale
        b2 = np.zeros((hidden_dim,))
        
        W3 = np.random.randn(hidden_dim, num_classes) * weight_scale
        b3 = np.zeros((num_classes))
        
        self.params['W1'], self.params['b1'] = W1, b1
        self.params['W2'], self.params['b2'] = W2, b2
        self.params['W3'], self.params['b3'] = W3, b3
        
    def loss(self, X, y):
        W1, b1 = self.params['W1'], self.params['b1']
        W2, b2 = self.params['W2'], self.params['b2']
        W3, b3 = self.params['W3'], self.params['b3']
        
        X = np.pad(X, pad_width=((0,), (0,), (self.pad,), (self.pad,)), mode='constant', constant_values=(0,))
        feature_map = np.zeros((X.shape[0], self.filter_num, self.feature_map_h, 
                                                             self.feature_map_w)) # with shape (H2, W2)
    
        # forward pass

        # conv layer:
        # todo: is there more efficient way?
        for i in range(self.feature_map_h):
            for j in range(self.feature_map_w):
                for k in range(self.filter_num):
                    feature_map[:,k,i,j] = np.sum(X[:, :, i*self.stride: i*self.stride + self.filter_size, 
                                                         j*self.stride: j*self.stride + self.filter_size]
                                                  * W1[k,:,:,:], axis=(1, 2, 3))
        feature_map += b1[None, :, None, None]
        
        # relu:
        feature_map[feature_map < 0] = 0
        
        # max pooling
        pooled_feature_map = np.zeros((X.shape[0], self.filter_num, self.pooled_feature_map_h, 
                                                                    self.pooled_feature_map_w))
        for i in range(self.pooled_feature_map_h):
            for j in range(self.pooled_feature_map_w):
                pooled_feature_map[:, :, i, j] = np.max(feature_map[:, :, i*self.pool_size: (i+1)*self.pool_size, 
                                                                          j*self.pool_size: (j+1)*self.pool_size], 
                                                        axis=(2, 3))
                
        # first fully connected layer
        scores1 = pooled_feature_map.reshape((X.shape[0], -1))
        scores2 = scores1.dot(W2) + b2
        
        # relu
        scores2[scores2 < 0] = 0
        
        # second fully connected layer
        scores = scores2.dot(W3) + b3
        
        # softmax
        exp_scores = np.exp(scores)
        sum_exp_scores = np.sum(exp_scores, axis=1)
        
        loss = np.sum(np.log(sum_exp_scores) - scores[range(scores.shape[0]), y])
        loss = loss/X.shape[0]
        loss += 0.5 * reg* (np.sum(W1*W1) + np.sum(W2*W2) + np.sum(W3*W3))
        
        # Backpropagation
        